In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
df = pd.read_csv('./data/medicare_partd_20162020.csv')

In [5]:
X=df.drop(columns = ['Avg_Spnd_Per_Bene_2020','Avg_Spnd_Per_Clm_2020','Avg_Spnd_Per_Dsg_Unt_Wghtd_2020','Tot_Spndng_2020'])
y=df['Avg_Spnd_Per_Bene_2020']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=20)

In [7]:
X_train

,Brnd_Name,Gnrc_Name,Tot_Mftr,Mftr_Name,Tot_Spndng_2016,Tot_Dsg_Unts_2016,Tot_Clms_2016,Tot_Benes_2016,Avg_Spnd_Per_Dsg_Unt_Wghtd_2016,Avg_Spnd_Per_Clm_2016,...,Avg_Spnd_Per_Dsg_Unt_Wghtd_2019,Avg_Spnd_Per_Clm_2019,Avg_Spnd_Per_Bene_2019,Outlier_Flag_2019,Tot_Dsg_Unts_2020,Tot_Clms_2020,Tot_Benes_2020,Outlier_Flag_2020,Chg_Avg_Spnd_Per_Dsg_Unt_19_20,CAGR_Avg_Spnd_Per_Dsg_Unt_16_20
9399,Oxycodone-Acetaminophen,Oxycodone HCl/Acetaminophen,1,Par Pharm.,5.528415e+05,3.569290e+05,6823.0,3278.0,1.548884,81.026157,...,1.519781,69.434662,145.766094,0.0,3.014190e+05,6603,3016.0,0,-0.033193,-0.013095
11290,Sodium Citrate-Citric Acid,Citric Acid/Sodium Citrate,1,Pharmaceutical,5.497508e+04,2.337012e+06,2105.0,656.0,0.023524,26.116428,...,0.029858,39.411315,129.821426,0.0,2.282016e+06,1650,546.0,0,-0.036825,0.051513
6283,Irbesartan,Irbesartan,1,Aurobindo Pharm,2.069771e+04,5.772350e+04,1487.0,409.0,0.358305,13.919106,...,0.445919,20.232244,56.920668,0.0,3.710050e+04,740,231.0,0,0.382542,0.145303
2331,Cinacalcet HCl,Cinacalcet HCl,1,Mylan,NaN,NaN,NaN,NaN,NaN,NaN,...,24.628118,787.235339,1678.663591,0.0,3.458400e+04,1034,384.0,0,-0.019516,-0.019516
991,Atorvastatin Calcium,Atorvastatin Calcium,1,Biocon Pharma I,NaN,NaN,NaN,NaN,NaN,NaN,...,0.219857,17.523084,37.587314,0.0,4.865988e+08,5819453,1772433.0,0,-0.210318,-0.171871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9620,Pen Needles,"Pen Needle, Diabetic",1,Owen Mumford Us,1.796044e+05,8.884150e+05,8331.0,2641.0,0.202155,21.558569,...,0.213503,21.984761,66.405254,0.0,5.038910e+05,4775,1518.0,0,0.006012,0.015267
7068,Lincocin,Lincomycin HCl,1,Pharmaci/Pfizer,1.747643e+04,1.272000e+03,221.0,161.0,13.739332,79.078869,...,20.477436,96.385172,121.529130,0.0,1.960000e+02,35,26.0,1,-0.321398,0.002839
7391,Magellan Insulin Syringe,"Syringe,Needle,Insuln,Sf 0.5ml",1,Overall,8.672061e+04,1.741850e+05,2943.0,574.0,0.497865,29.466738,...,0.406825,33.769795,83.230606,0.0,1.470000e+04,162,54.0,0,-0.035623,-0.057816
4367,Erythromycin-Benzoyl Peroxide,Erythromycin/Benzoyl Peroxide,3,Overall,4.296319e+06,8.480327e+05,19494.0,8176.0,5.065248,220.391882,...,3.656690,162.176400,418.674781,0.0,4.629815e+05,10509,3832.0,0,-0.116191,-0.106259


## Set a Baseline

In [8]:
y_train.mean()

7709.58844571059

In [11]:
baseline = np.full_like(y_test, y_train.mean())
baseline

array([7709.58844571, 7709.58844571, 7709.58844571, ..., 7709.58844571,
       7709.58844571, 7709.58844571])

In [13]:
len(baseline), len(y_test)

(3393, 3393)

## Create a basic Regression Model

#### first, encode the data

In [27]:
transf = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse=False),['Brnd_Name', 'Gnrc_Name','Mftr_Name']),
    remainder='passthrough',
    n_jobs=-1,
    verbose_feature_names_out=False
)

In [33]:
transf.fit(X_train)

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['Brnd_Name', 'Gnrc_Name', 'Mftr_Name'])],
                  verbose_feature_names_out=False)

In [35]:
enc_column_names = transf.get_feature_names_out()
enc_column_names

array(['Brnd_Name_1st Tier Unifine Pentips',
       'Brnd_Name_1st Tier Unifine Pentips Plus', 'Brnd_Name_Abacavir',
       ..., 'Outlier_Flag_2020', 'Chg_Avg_Spnd_Per_Dsg_Unt_19_20',
       'CAGR_Avg_Spnd_Per_Dsg_Unt_16_20'], dtype=object)

In [34]:
X_train_enc = pd.DataFrame(transf.transform(X_train), columns= enc_column_names)
X_test_enc = pd.DataFrame(transf.transform(X_test), columns= enc_column_names)

In [36]:
lr = LinearRegression()

In [37]:
lr.fit(X_train_enc, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').